# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [2]:
# Add as many code cells as you need
df = pd.read_csv('milestone_1_zillow.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77070 entries, 0 to 77069
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   bathroomcnt                       77070 non-null  int64  
 1   bedroomcnt                        77070 non-null  int64  
 2   buildingqualitytypeid             77070 non-null  float64
 3   calculatedfinishedsquarefeet      77070 non-null  float64
 4   fireplacecnt                      77070 non-null  int64  
 5   garagecarcnt                      77070 non-null  int64  
 6   garagetotalsqft                   77070 non-null  float64
 7   lotsizesquarefeet                 77070 non-null  float64
 8   roomcnt                           77070 non-null  int64  
 9   threequarterbathnbr               77070 non-null  int64  
 10  unitcnt                           77070 non-null  int64  
 11  yearbuilt                         77070 non-null  float64
 12  numb

In [3]:
# train_test_split set up 
X = df.drop(columns=['taxvaluedollarcnt'])
y = df['taxvaluedollarcnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((61656, 40), (15414, 40), (61656,), (15414,))

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
X_train_scaled.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,roomcnt,threequarterbathnbr,...,heatingorsystemtypeid_24.0,regionidcounty_1286.0,regionidcounty_2061.0,regionidcounty_3101.0,log_garagetotalsqft,log_lotsizesquarefeet,log_calculatedfinishedsquarefeet,calc_bedbath,prod_bathroomcnt_bedroomcnt,prod_fireplacecnt_garagecarcnt
0,-0.279573,-0.060467,0.610263,-0.081186,-0.181079,0.449537,0.536291,-0.252031,1.590810,-0.026946,...,-0.104339,1.678462,-0.29788,-1.382556,0.547220,-0.199303,0.140107,-0.379626,-0.270728,0.090455
1,-0.279573,-0.060467,0.610263,-0.494301,-0.181079,0.449537,0.649533,-0.279582,1.590810,-0.026946,...,-0.104339,1.678462,-0.29788,-1.382556,0.580913,-0.524299,-0.440730,-0.379626,-0.270728,0.090455
2,-0.279573,-0.060467,-0.677032,-0.472170,-0.181079,0.449537,0.029105,-0.271632,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.360377,-0.414883,-0.405477,-0.379626,-0.270728,0.090455
3,0.730112,1.710615,-0.677032,0.527948,-0.181079,0.449537,0.765593,-0.243554,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.613251,-0.122653,0.789979,-0.609151,1.180850,0.090455
4,-0.279573,-0.060467,0.610263,0.308744,-0.181079,0.449537,1.318369,-0.210376,-0.525373,-0.026946,...,-0.104339,1.678462,-0.29788,-1.382556,0.743898,0.118019,0.577233,-0.379626,-0.270728,0.090455


In [5]:
X_test_scaled.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,roomcnt,threequarterbathnbr,...,heatingorsystemtypeid_24.0,regionidcounty_1286.0,regionidcounty_2061.0,regionidcounty_3101.0,log_garagetotalsqft,log_lotsizesquarefeet,log_calculatedfinishedsquarefeet,calc_bedbath,prod_bathroomcnt_bedroomcnt,prod_fireplacecnt_garagecarcnt
0,-0.279573,-0.060467,-0.677032,-0.254020,-0.181079,0.449537,0.033489,-0.277522,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.362329,-0.494350,-0.084838,-0.379626,-0.270728,0.090455
1,-0.279573,-0.060467,-0.677032,0.044224,-0.181079,0.449537,0.008199,-0.243537,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.350963,-0.122511,0.290229,-0.379626,-0.270728,0.090455
2,-0.279573,-0.060467,0.610263,-0.305660,-0.181079,-1.365449,-1.095103,-0.308006,-0.525373,-0.026946,...,-0.104339,1.678462,-0.29788,-1.382556,-2.539603,-1.151992,-0.156688,-0.379626,-0.270728,-0.768753
3,-1.289257,-0.946007,-1.320680,-0.160226,-0.181079,-1.365449,-0.506671,-0.267224,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.036118,-0.360588,0.040036,-0.953437,-0.915874,-0.768753
4,-0.279573,-0.946007,-0.033385,-0.503786,-0.181079,-1.365449,-0.296692,-0.136020,-0.525373,-0.026946,...,-0.104339,-0.595783,-0.29788,0.723298,0.188854,0.475032,-0.456011,0.767997,-0.593301,-0.768753


In [6]:
selected_features = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet',
       'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
       'lotsizesquarefeet', 'roomcnt', 'threequarterbathnbr', 'unitcnt',
       'yearbuilt'] #if we want to subset by the best features identfied by the forward feature selection in milestone 1
X_train_scaled_selected = X_train_scaled[selected_features]
X_test_scaled_selected = X_test_scaled[selected_features]
X_train_scaled_selected.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,roomcnt,threequarterbathnbr,unitcnt,yearbuilt
0,-0.279573,-0.060467,-0.081186,-0.181079,0.449537,0.536291,-0.252031,1.590810,-0.026946,-0.070276,-0.401958
1,-0.279573,-0.060467,-0.494301,-0.181079,0.449537,0.649533,-0.279582,1.590810,-0.026946,-0.070276,-0.317900
2,-0.279573,-0.060467,-0.472170,-0.181079,0.449537,0.029105,-0.271632,-0.525373,-0.026946,-0.070276,-0.864274
3,0.730112,1.710615,0.527948,-0.181079,0.449537,0.765593,-0.243554,-0.525373,-0.026946,0.912472,-0.822246
4,-0.279573,-0.060467,0.308744,-0.181079,0.449537,1.318369,-0.210376,-0.525373,-0.026946,-0.070276,1.783539


In [7]:
X_test_scaled_selected.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplacecnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,roomcnt,threequarterbathnbr,unitcnt,yearbuilt
0,-0.279573,-0.060467,-0.254020,-0.181079,0.449537,0.033489,-0.277522,-0.525373,-0.026946,-0.070276,-1.032389
1,-0.279573,-0.060467,0.044224,-0.181079,0.449537,0.008199,-0.243537,-0.525373,-0.026946,-0.070276,-0.612102
2,-0.279573,-0.060467,-0.305660,-0.181079,-1.365449,-1.095103,-0.308006,-0.525373,-0.026946,-0.070276,0.900934
3,-1.289257,-0.946007,-0.160226,-0.181079,-1.365449,-0.506671,-0.267224,-0.525373,-0.026946,-0.070276,-1.914994
4,-0.279573,-0.946007,-0.503786,-0.181079,-1.365449,-0.296692,-0.136020,-0.525373,-0.026946,-0.070276,-0.275871


### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Bagging': BaggingRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

# Set up cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# RMSE scorer (neg_root_mean_squared_error because sklearn minimizes loss)
scorer = make_scorer(mean_squared_error, squared=False)

# Store results
results = []

for name, model in models.items():
    # Now using n_jobs=-1 to parallelize cross-validation internally
    scores = cross_val_score(model, X_train_scaled, y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
    rmse_scores = -scores  # convert to positive RMSE

    results.append({
        'Model': name,
        'Mean RMSE': np.mean(rmse_scores),
        'Std RMSE': np.std(rmse_scores)
    })

# Results into a dataframe
results_df = pd.DataFrame(results)

# Format Mean RMSE and Std RMSE as dollars
results_df['Mean RMSE'] = results_df['Mean RMSE'].apply(lambda x: f"${x:,.0f}")
results_df['Std RMSE'] = results_df['Std RMSE'].apply(lambda x: f"${x:,.0f}")

# Display the formatted results
print(results_df)


In [9]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Bagging': BaggingRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

# Set up cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# RMSE scorer (neg_root_mean_squared_error because sklearn minimizes loss)
scorer = make_scorer(mean_squared_error, squared=False)

# Store results
results = []

for name, model in models.items():
    # Now using n_jobs=-1 to parallelize cross-validation internally
    scores = cross_val_score(model, X_train_scaled_selected, y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
    rmse_scores = -scores  # convert to positive RMSE

    results.append({
        'Model': name,
        'Mean RMSE': np.mean(rmse_scores),
        'Std RMSE': np.std(rmse_scores)
    })

# Results into a dataframe
results_df = pd.DataFrame(results)

print(results_df)

               Model      Mean RMSE      Std RMSE
0  Linear Regression  510949.737848  51662.184047
1   Ridge Regression  510942.307207  51650.621548
2   Lasso Regression  510947.161545  51658.220399
3      Decision Tree  642205.660404  32975.508954
4            Bagging  493242.117226  29908.742555
5      Random Forest  474455.223347  30925.431682
6  Gradient Boosting  472961.443995  31596.509132


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> Your text here

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [10]:
# Add as many code cells as you need

### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



> Your text here

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [11]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here